In [ ]:
# -*- coding: utf-8 -*-
import csv, os, time, random, sys, re
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

START_URL = "https://sugang.hongik.ac.kr/cn50000.jsp"

DAY_PATTERN = re.compile(r"[월화수목금토일]")
INT_PATTERN = re.compile(r"^\d+$")
TIME_PATTERN = re.compile(r"[월화수목금토일]\s*\d+")  # 월234, 화 234 등

INVALID_PROF = {"전체", "없음", "미정", "-", ""}

# ---------------- DB 연결 ----------------
def db_connect():
    return pymysql.connect(
        host="localhost",
        user="root",
        password="4205",
        database="qnet_crawling2",
        charset="utf8mb4",
        autocommit=True
    )

def save_course(row, conn):
    sql = """
        INSERT INTO course (opened_grade, department, course_name, enrolled, professor, schedule)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE
            enrolled = VALUES(enrolled),
            professor = VALUES(professor),
            schedule = VALUES(schedule);
    """
    with conn.cursor() as cur:
        cur.execute(sql, (
            row.get("개설학년"),
            row.get("주관학과"),
            row.get("과목명"),
            int(row.get("수강인원") or 0),
            row.get("교수명"),
            row.get("요일및시간"),
        ))

# ---------------- 유틸 ----------------
def build_driver():
    opt = webdriver.ChromeOptions()
    opt.add_argument("--start-maximized")
    opt.add_argument("--disable-gpu")
    return webdriver.Chrome(options=opt)

def norm(s: str) -> str:
    return " ".join((s or "").replace("\xa0", " ").split())

def wait_css(driver, css, timeout=15):
    return WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css))
    )

# ---------------- 테이블 파싱 ----------------
def parse_table_rows(table):
    rows = table.find_elements(By.CSS_SELECTOR, "tr")[1:]
    out = []

    for r in rows:
        tds = r.find_elements(By.CSS_SELECTOR, "td")
        if not tds:
            continue

        # 과목(list_subject) 위치 찾기
        subj_idx = -1
        for i, td in enumerate(tds):
            if "list_subject" in (td.get_attribute("class") or ""):
                subj_idx = i
                break
        if subj_idx < 0:
            continue

        course_name = norm(tds[subj_idx].get_attribute("innerText"))
        opened_grade = norm(tds[0].get_attribute("innerText"))

        # 🔵 주관학과: 과목 td 기준 왼쪽 4칸
        try:
            dept_text = norm(tds[subj_idx - 4].get_attribute("innerText"))
        except Exception:
            dept_text = ""

        # ---------------- 요일/시간 + 교수명 패턴 기반 추출 ----------------
        time_idx, time_text = None, ""
        prof_text = ""
        prof_candidates = []
        saw_all = False  # '전체' 본 적 있는지

        for j in range(subj_idx + 1, len(tds)):
            txt = norm(tds[j].get_attribute("innerText"))
            if not txt:
                continue

            # 요일/시간인지 먼저 체크
            if TIME_PATTERN.search(txt) or (DAY_PATTERN.search(txt) and any(ch.isdigit() for ch in txt)):
                if time_idx is None:  # 첫 번째로 발견된 시간 셀만 사용
                    time_idx, time_text = j, txt

            # 교수 후보 수집
            # 숫자만, 시간패턴, 완전 의미없는 값은 제외
            if INT_PATTERN.match(txt):
                continue
            if TIME_PATTERN.search(txt):
                continue
            if txt in INVALID_PROF:
                saw_all = True
                continue

            # 한글/영문자가 섞인 텍스트는 교수 이름 후보로 본다
            if re.search(r"[가-힣A-Za-z]", txt):
                prof_candidates.append((j, txt))

        # 교수명 확정: 시간 셀 이전의 마지막 후보
        if prof_candidates:
            if time_idx is not None:
                prof_candidates = [(idx, t) for idx, t in prof_candidates if idx < time_idx]
            if prof_candidates:
                prof_text = prof_candidates[-1][1]
        # 후보가 전혀 없고 '전체'만 있었으면 '전체'라도 넣어준다
        if not prof_text and saw_all:
            prof_text = "전체"

        # ---------------- 수강인원 ----------------
        enrolled_text = ""
        if time_idx is not None:
            scan_end = time_idx - 1
        else:
            scan_end = len(tds) - 1

        for j in range(scan_end, subj_idx, -1):
            txt = norm(tds[j].get_attribute("innerText"))
            if INT_PATTERN.match(txt):
                enrolled_text = txt
                break

        item = {
            "개설학년": opened_grade,
            "주관학과": dept_text,
            "과목명": course_name,
            "수강인원": enrolled_text,
            "교수명": prof_text,
            "요일및시간": time_text,
        }

        if any(item.values()):
            out.append(item)

    return out

# ---------------- 팝업 파싱 ----------------
def parse_second_page(driver, timeout=15):
    end = time.time() + timeout
    while time.time() < end:
        try:
            driver.switch_to.default_content()
            tables = driver.find_elements(By.CSS_SELECTOR, "#select_list")
            if tables:
                return parse_table_rows(tables[0])

            for f in driver.find_elements(By.TAG_NAME, "iframe"):
                try:
                    driver.switch_to.frame(f)
                    tables = driver.find_elements(By.CSS_SELECTOR, "#select_list")
                    if tables:
                        return parse_table_rows(tables[0])
                finally:
                    driver.switch_to.default_content()

        except Exception:
            pass
        time.sleep(0.4)
    return []

# ---------------- main ----------------
def main():
    conn = db_connect()
    driver = build_driver()
    driver.get(START_URL)

    wait_css(driver, "#table_seoul")

    anchors = driver.find_elements(By.CSS_SELECTOR, '#table_seoul a[href^="javascript:gocn4001"]')

    out_path = "courses.csv"
    need_header = not os.path.exists(out_path)
    f = open(out_path, "a", newline="", encoding="utf-8-sig")
    writer = csv.DictWriter(f, fieldnames=["개설학년", "주관학과", "과목명", "수강인원", "교수명", "요일및시간"])
    if need_header:
        writer.writeheader()

    original = driver.current_window_handle

    for idx, a in enumerate(anchors, 1):
        print(f"[INFO] ({idx}/{len(anchors)}) 링크 클릭…")

        before = set(driver.window_handles)
        driver.execute_script("arguments[0].scrollIntoView({block:'center'})", a)
        time.sleep(0.2)
        a.click()
        time.sleep(0.6)

        after = set(driver.window_handles)

        try:
            if len(after) > len(before):
                new_handler = list(after - before)[0]
                driver.switch_to.window(new_handler)

                rows = parse_second_page(driver)
                for row in rows:
                    print("   ", row)
                    writer.writerow(row)
                    save_course(row, conn)

                driver.close()
                driver.switch_to.window(original)

            else:
                rows = parse_second_page(driver)
                for row in rows:
                    print("   ", row)
                    writer.writerow(row)
                    save_course(row, conn)

                driver.back()
                wait_css(driver, "#table_seoul")
                anchors = driver.find_elements(By.CSS_SELECTOR, '#table_seoul a[href^="javascript:gocn4001"]')

        except TimeoutException:
            print("  [TIMEOUT] 건너뜀")

        time.sleep(random.uniform(0.6, 1.2))

    conn.close()
    f.close()
    driver.quit()
    print("[DONE] CSV + DB 저장 완료!")

if __name__ == "__main__":
    main()
